**Tabla de contenido**

- [Pronóstico del uso de ancho de banda para centros de datos](#Pronostico-del-uso-de-ancho-de-banda-para-centros-de-datos)
- [Examinando el proceso de media movil autorregresivo](#Examinando-el-proceso-de-media-movil-autorregresivo)
- [Identificación de un proceso ARMA estacionario](#Identificacion-de-un-proceso-ARMA-estacionario)
- [Elaboración de un procedimiento de modelado general](#Elaboracion-de-un-procedimiento-de-modelado-general)
- [Comprendiendo el criterio de información de Akaike (AIC)](#Comprendiendo-el-criterio-de-informacion-de-Akaike-(AIC))

En este capítulo, examinaremos el `proceso de media móvil autorregresiva`, ARMA(p,q), donde p denota el orden de la parte autorregresiva y q denota el orden de la parte de media móvil. Además, el uso de los gráficos ACF y PACF para determinar los órdenes q y p, respectivamente, se vuelve complicado, ya que ambos gráficos mostrarán un patrón de decaimiento lento o sinusoidal. Por lo tanto, definiremos un procedimiento de modelado general que nos permitirá modelar series temporales tan complejas. Este procedimiento implica la selección del modelo utilizando el criterio de información de Akaike (AIC), que determinará la combinación óptima de p y q para nuestra serie.

Entonces debemos evaluar la validez del modelo utilizando el análisis de residuos, estudiando el correlograma, el diagrama Q-Q y el diagrama de densidad de los residuos del modelo para evaluar si se asemejan estrechamente al ruido blanco. Si ese es el caso, podemos pasar a pronosticar nuestra serie temporal utilizando el modelo ARMA(p,q).

Este capítulo introducirá conocimientos fundamentales para la previsión de series temporales complejas. Todos los conceptos introducidos aquí se volverán a utilizar en capítulos posteriores cuando empecemos a modelar series temporales no estacionarias e incorporar estacionalidad y variables exógenas.

# Pronostico del uso de ancho de banda para centros de datos

Supongamos que se te encarga predecir el uso del ancho de banda para un gran centro de datos. El ancho de banda se define como la tasa máxima de datos que se puede transferir. Su unidad base es bits por segundo (bps).

Prever el uso del ancho de banda permite a los centros de datos gestionar mejor sus recursos informáticos. En el caso de que se espere un menor uso del ancho de banda, pueden apagar algunos de sus recursos informáticos. Esto, a su vez, reduce los gastos y permite el mantenimiento. Por otro lado, si se espera que el uso del ancho de banda aumente, pueden dedicar los recursos necesarios para satisfacer la demanda y garantizar baja latencia, manteniendo así satisfechos a sus clientes.

Para esta situación, hay 10,000 puntos de datos que representan el uso de ancho de banda por hora comenzando el 1 de enero de 2019. Aquí el ancho de banda se mide en megabits por segundo (Mbps).

Para predecir el uso del ancho de banda, necesitamos identificar el proceso subyacente en nuestra serie. Así, seguiremos los pasos que definimos en el capítulo 5. De esa manera, podemos verificar si tenemos un paseo aleatorio, un proceso de media móvil o un proceso autorregresivo. 

`El primer paso es recopilar los datos`, lo cual ya se ha hecho en este caso. Luego debemos determinar si nuestra serie es estacionaria o no. La presencia de una tendencia en el gráfico sugiere que nuestra serie no es estacionaria. Sin embargo, aplicaremos la prueba ADF para verificar la estacionariedad y aplicaremos una transformación en consecuencia.

`Luego trazaremos la función ACF` y descubriremos que hay coeficientes de autocorrelación significativos después del rezago 0, lo que significa que no es una caminata aleatoria. Sin embargo, observaremos que los coeficientes decayen lentamente. No se vuelven abruptamente no significativos después de cierto rezago, lo que significa que no es un proceso de media móvil pura.

`Luego pasaremos a trazar la función PACF`. Esta vez notaremos un patrón sinusoidal, lo que significa que los coeficientes no se vuelven abruptamente no significativos después de un cierto rezago. Esto nos llevará a la conclusión de que tampoco se trata de un proceso puramente autorregresivo.

Por lo tanto, debe ser una combinación de procesos autorregresivos y de media móvil, lo que resulta en un proceso de media móvil autorregresiva que se puede modelar con el modelo ARMA(p,q), donde p es el orden del proceso autorregresivo y q es el orden del proceso de media móvil. Es difícil utilizar los gráficos ACF y PACF para encontrar respectivamente p y q, por lo que ajustaremos muchos modelos ARMA(p,q) con diferentes combinaciones de valores para p y q. Luego seleccionaremos un modelo de acuerdo con el criterio de información de Akaike y evaluaremos su viabilidad analizando sus residuos.

`Idealmente, los residuos de un modelo tendrán características similares al ruido blanco`. Entonces podremos utilizar este modelo para hacer pronósticos. Para este ejemplo, pronosticaremos el uso de ancho de banda por hora durante las próximas dos horas.


# Examinando el proceso de media movil autorregresivo

El `proceso autorregresivo de media móvi`l` es una combinación del `proceso autorregresivo` y el `proceso de media móvil`. Establece que el valor presente es linealmente dependiente de sus propios valores anteriores y una constante, al igual que en un proceso autorregresivo, así como de la media de la serie, el término de error actual y los términos de error pasados, como en un proceso de media móvil.

El proceso de media móvil autorregresivo se denota como `ARMA(p,q)`, donde $p$ es `el orden de la porción autorregresiva` y $q$ es el orden de la porción de media móvil. Matemáticamente, el proceso ARMA(p,q) se expresa como una combinación lineal de una constante $C$, los valores pasados de la serie $y_{t–p}$, la media de la serie $µ$, los términos de error pasados $ϵ_{t–q}$ y el término de error actual $ϵ_t$, como se muestra en la siguiente ecuanción. $$y_t = C + ϕ_1y_{t–1} + ϕ_2y_{t–2} +⋅⋅⋅+ ϕ_py_{t–p} + ϵ_t + θ_1ϵ_{t–1} + θ_2ϵ_{t–2} +⋅⋅⋅+ θ_qϵ_{t–q}$$

Nuevamente, el orden p determina el número de valores pasados que afectan el valor presente. De manera similar, el orden q determina el número de términos de error pasados que afectan el valor presente. En otras palabras, los órdenes p y q dictan el número de parámetros para las porciones autorregresivas y de media móvil, respectivamente.

Así, `si tenemos un proceso ARMA(1,1)`, estamos combinando un `proceso autorregresivo de orden 1`, o AR(1), con un proceso de `media móvil de orden 1`, o MA(1). Recuerde que un proceso autorregresivo de primer orden es una combinación lineal de una constante $C$, el valor de la serie en el paso de tiempo anterior $ϕ_1y_{t–1}$, y ruido blanco $ϵ_t$, como se muestra en la siguiente ecuanción.$$AR(1) := y_t = C + ϕ_1y_{t–1} + ϵ_t$$
También recuerda que un proceso de media móvil de primer orden, o MA(1), es una combinación lineal de la media de la serie $μ$, el término de error actual $ϵ_t$ y el término de error en el tiempo anterior $θ_1ϵ_{t–1}, como se muestra en la siguiente ecuanción. $$MA(1) := y_t = μ + ϵ_t + θ_1ϵ_{t–1}$$

Podemos combinar los procesos AR(1) y MA(1) para obtener un proceso ARMA(1,1) como se muestra en la siguiente ecuación. 
$$ARMA(1,1) := y_t = C + ϕ_1y_{t–1} + ϵ_t + θ_1ϵ_{t–1}$$

En el caso donde p = 0, tenemos un proceso ARMA(0,q), que es equivalente a un proceso puro MA(q) como se vio en el capítulo 4. De manera similar, si q = 0, tenemos un proceso ARMA(p,0), que es equivalente a un proceso puro AR(p), como se vio en el capítulo 5.

Ahora podemos ver cómo el orden p solo afecta la porción autorregresiva del proceso al determinar el número de valores pasados que se incluyen en la ecuación. De manera similar, el orden q solo afecta la porción de media móvil del proceso al determinar el número de términos de error pasados que se incluyen en la ecuación de ARMA(p,q). Por supuesto, cuanto más altos sean los órdenes p y q, más términos se incluirán, y más complejo se vuelve nuestro proceso.

Para modelar y pronosticar un proceso ARMA(p,q), necesitamos encontrar los órdenes p y q. De esa manera, podemos utilizar un modelo ARMA(p,q) para ajustar los datos disponibles y producir pronósticos.

# Identificacion de un proceso ARMA estacionario

Ahora que hemos definido el proceso autorregresivo de media móvil y hemos visto cómo los órdenes p y q afectan la ecuación del modelo, necesitamos determinar cómo identificar un proceso subyacente así en una serie temporal dada.

`Si tu proceso es estacionario y tanto los gráficos de ACF como de PACF muestran un patrón decreciente o sinusoidal, entonces es un proceso ARMA(p,q) estacionario.`

Sabemos que determinar el orden de nuestro proceso es clave en la modelización y la previsión, ya que el orden dictará cuántos parámetros deben incluirse en nuestro modelo. `Dado que los gráficos de ACF y PACF no son útiles en el caso de un proceso ARMA(p,q)`, debemos, por lo tanto, idear un procedimiento de modelización general que nos permita encontrar la combinación adecuada de (p,q) para nuestro modelo.

# Elaboracion de un procedimiento de modelado general

`Procedimiento general de modelado para un proceso ARMA(p, q)`. Los primeros pasos son recopilar los datos, probar la estacionariedad y aplicar transformaciones en consecuencia. Luego definimos una lista de posibles valores para p y q. A continuación, ajustamos cada combinación de ARMA(p, q) a nuestros datos y seleccionamos el modelo con el AIC más bajo. Luego realizamos el análisis de residuos observando el gráfico Q-Q y el correlograma de residuos. Si se acercan al de ruido blanco, el modelo se puede utilizar para pronósticos. De lo contrario, debemos probar diferentes valores para p y q.

Los primeros pasos siguen siendo los mismos que hemos ido construyendo gradualmente hasta el capítulo 5, ya que todavía debemos recopilar los datos, probar la estacionaridad y aplicar transformaciones de acuerdo con ello. Luego enumeramos diferentes valores posibles de p y q; tenga en cuenta que solo toman enteros positivos. Con una lista de valores posibles, podemos ajustar cada combinación única de ARMA(p,q) a nuestros datos.

Una vez hecho esto, podemos calcular el criterio de información de Akaike (AIC). Esto cuantifica la calidad de cada modelo en relación entre sí. El modelo con el AIC más bajo es entonces seleccionado.

A partir de ahí, podemos analizar los residuos del modelo, que son la diferencia entre los valores reales y los valores predichos del modelo. Idealmente, los residuos deberían parecer ruido blanco, lo que significaría que cualquier diferencia entre los valores predichos y los valores reales se debe a la aleatoriedad. Por lo tanto, los residuos deben ser no correlacionados y distribuidos independientemente.

Podemos evaluar esas propiedades estudiando el gráfico cuantil-cuantil (gráfico Q-Q) y realizando la prueba de Ljung-Box. Si el análisis nos lleva a concluir que los residuales son completamente aleatorios, tenemos un modelo listo para pronosticar. De lo contrario, debemos probar un conjunto diferente de valores para p y q y comenzar el proceso nuevamente.

# Comprendiendo el criterio de informacion de Akaike (AIC)

Antes de cubrir los pasos descritos anteriormente, `necesitamos determinar cómo elegiremos el mejor modelo de todos los modelos que ajustaremos`. Aquí utilizaremos el `criterio de información de Akaike (AIC)` para seleccionar el modelo óptimo.

`El AIC estima la calidad de un modelo en relación con otros modelos`. Dado que habrá algo de información perdida cuando un modelo se ajusta a los datos, `el AIC cuantifica la cantidad relativa de información perdida por el modelo`. Cuanta menos información se pierda, menor será el valor del AIC y mejor será el modelo.

El AIC es una función del número de parámetros estimados k y el valor máximo de la función de verosimilitud para el modelo L̂ , como se muestra en la siguiente ecuación $$AIC 2k – 2ln(L̂ )$$

El número de parámetros estimados k está directamente relacionado con el orden (p,q) de un modelo ARMA(p,q). Si ajustamos un modelo ARMA(2,2), entonces tenemos 2 + 2 = 4 parámetros para estimar. Si ajustamos un modelo ARMA(3,4), entonces tenemos 3 + 4 = 7 parámetros para estimar. Puedes ver cómo ajustar un modelo más complejo puede penalizar la puntuación AIC: a medida que el orden (p,q) aumenta, el número de parámetros k aumenta, y por lo tanto, el AIC aumenta.